# 2. Data Analysis


## 🛠️ Design Decisions & Data Cleaning

- **Database schema**: A single `movies` table stores core metadata (`movie_id`, `title`, `year`, `vote_average`, etc.).

  - List-type fields like `genre_ids` and `genres` are stored as JSON strings for flexibility.
  - `created_at` adds a timestamp for traceability.

- **Storage format**: Cleaned data is saved to a local **SQLite** database (`data/database.db`) for reproducibility and easy querying.

- **Cleaning steps**:

  - Loaded raw JSON into a `pandas` DataFrame.
  - Converted list columns (`genre_ids`, `genres`) to JSON strings to preserve structure.
  - Ensured output directory exists and overwrote existing data with `if_exists='replace'`.

- **Validation**: A verification function checks total row count and displays sample entries to confirm successful storage.


In [30]:
import os
import json
import sqlite3
import pandas as pd

In [31]:
DB_PATH = "../data/database.db"
RAW_DIR = "../data/raw/"

def create_database_schema(db_path):
    """Create the database schema for movies"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Create movies table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS movies (
            movie_id INTEGER PRIMARY KEY,
            title TEXT NOT NULL,
            year INTEGER NOT NULL,
            popularity REAL,
            vote_average REAL,
            vote_count INTEGER,
            genre_ids TEXT,  -- Store as JSON string
            genres TEXT,     -- Store as JSON string
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    conn.commit()
    conn.close()
    print(f"✅ Database schema created at {db_path}")

def save_to_sqlite(df, db_path):
    """Save DataFrame to SQLite database"""
    # Ensure the directory exists
    os.makedirs(os.path.dirname(db_path), exist_ok=True)
    
    # Create database schema
    create_database_schema(db_path)
    
    # Convert list columns to JSON strings for SQLite storage
    df_copy = df.copy()
    df_copy['genre_ids'] = df_copy['genre_ids'].apply(json.dumps)
    df_copy['genres'] = df_copy['genres'].apply(json.dumps)
    
    # Save to SQLite
    conn = sqlite3.connect(db_path)
    df_copy.to_sql('movies', conn, if_exists='replace', index=False)
    conn.close()
    
    print(f"✅ Data saved to SQLite database: {db_path}")
    print(f"   Total records: {len(df)}")

# Load data from JSON file (assuming it was created by your data collection script)
json_file_path = f"{RAW_DIR}/movies_2020_2024.json"

# Read the JSON data into a DataFrame
df = pd.read_json(json_file_path)

# Save to SQLite database
save_to_sqlite(df, DB_PATH)

# Optional: Verify the data was saved correctly
def verify_database(db_path):
    """Verify the data was saved correctly"""
    conn = sqlite3.connect(db_path)
    
    # Get table info
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM movies")
    count = cursor.fetchone()[0]
    
    cursor.execute("SELECT * FROM movies LIMIT 3")
    sample_rows = cursor.fetchall()
    
    conn.close()
    
    print(f"\n📊 Database verification:")
    print(f"   Total movies in database: {count}")
    print(f"   Sample data (first 3 rows):")
    for i, row in enumerate(sample_rows, 1):
        print(f"   {i}. {row[1]} ({row[2]}) - Rating: {row[4]}")


✅ Database schema created at ../data/database.db
✅ Data saved to SQLite database: ../data/database.db
   Total records: 375


In [32]:
def create_movie_genres_table(df_exploded, db_path):
    """Create a separate table to store movie-genre relationships"""
    # Prepare the exploded DataFrame
    genre_df = df_exploded[['movie_id', 'genres']].copy()
    genre_df = genre_df.rename(columns={'genres': 'genre'})
    
    # Optional: Strip whitespace or handle malformed data
    genre_df['genre'] = genre_df['genre'].astype(str).str.strip()

    # Save to SQLite
    conn = sqlite3.connect(db_path)
    
    # Create table if not exists
    conn.execute('''
        CREATE TABLE IF NOT EXISTS movie_genres (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            movie_id INTEGER,
            genre TEXT,
            FOREIGN KEY(movie_id) REFERENCES movies(movie_id)
        );
    ''')
    
    # Replace data in table
    genre_df.to_sql('movie_genres', conn, if_exists='replace', index=False)
    conn.close()

    print(f"✅ Saved exploded genre data to 'movie_genres' table.")

DB_PATH = "../data/database.db"

# Step 1: Load and expand genres from SQLite
def load_movies_with_genres(db_path):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql("SELECT * FROM movies", conn)
    conn.close()
    
    # Parse JSON string into lists
    df['genres'] = df['genres'].apply(json.loads)

    # Explode genres into separate rows for analysis
    df_exploded = df.explode('genres')
    
    return df, df_exploded

df, df_exploded = load_movies_with_genres(DB_PATH)
create_movie_genres_table(df_exploded, DB_PATH)


✅ Saved exploded genre data to 'movie_genres' table.


In [33]:
# import sqlite3

# DB_PATH = "../data/database.db"  # update if needed

# def query_with_sqlite3():
#     """Using raw SQL queries with sqlite3"""
#     conn = sqlite3.connect(DB_PATH)
#     cursor = conn.cursor()
    
#     cursor.execute("SELECT COUNT(*) FROM movies")
#     total_movies = cursor.fetchone()[0]
#     print(f"Total number of movies in the database: {total_movies}")
    
#     cursor.execute("SELECT COUNT(DISTINCT movie_id) FROM movies")
#     unique_movies = cursor.fetchone()[0]
#     print(f"Total number of unique movies by movie_id: {unique_movies}")

#     cursor.execute("SELECT COUNT(DISTINCT title) FROM movies")
#     unique_titles = cursor.fetchone()[0]
#     print(f"Total number of unique movies by title: {unique_titles}")

#     # Movies with the lowest vote_count
#     cursor.execute("""
#         SELECT title, vote_count
#         FROM movies
#         WHERE vote_count = (SELECT MIN(vote_count) FROM movies)
#     """)
#     lowest_vote_movies = cursor.fetchall()
#     print("\nMovies with the lowest number of votes:")
#     for title, votes in lowest_vote_movies:
#         print(f"{title}: {votes}")

#     # Movies with the highest vote_count
#     cursor.execute("""
#         SELECT title, vote_count
#         FROM movies
#         WHERE vote_count = (SELECT MAX(vote_count) FROM movies)
#     """)
#     highest_vote_movies = cursor.fetchall()
#     print("\nMovie with the highest number of votes:")
#     for title, votes in highest_vote_movies:
#         print(f"{title}: {votes}")

#     conn.close()

# query_with_sqlite3()


In [34]:
def print_table_row_counts(db_path):
    """Print the number of rows in each table in the SQLite database"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Get all user-defined tables (ignore SQLite internal tables)
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
    tables = cursor.fetchall()

    print(f"\n📋 Row counts for tables in {db_path}:\n")
    for (table_name,) in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        count = cursor.fetchone()[0]
        print(f"   🗂 {table_name}: {count} rows")

    conn.close()
print_table_row_counts(DB_PATH)



📋 Row counts for tables in ../data/database.db:

   🗂 movies: 375 rows
   🗂 movie_genres: 1066 rows
